In [18]:
# Install Dependencies
!pip install streamlit google-generativeai PyPDF2 python-docx pillow SpeechRecognition gTTS pyaudio pyngrok ipywidgets -q

In [19]:
!pip install streamlit google-generativeai PyPDF2 python-docx pillow SpeechRecognition gTTS pyaudio pyngrok ipywidgets -q
!apt-get update && apt-get install -y portaudio19-dev flac -q


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [20]:
import streamlit as st
import os
import tempfile
import io
import base64
import json
import time
from typing import List, Optional

# Google Colab and Jupyter specific
from google.colab import output
from IPython.display import Javascript, HTML, display, Audio
import IPython.display as ipd

# Google Generative AI
import google.generativeai as genai

# Document processing
from PIL import Image
import PyPDF2
import docx

# Audio processing
import speech_recognition as sr
from gtts import gTTS
import numpy as np

print("✅ All modules imported successfully!")

✅ All modules imported successfully!


In [21]:
# Page configuration
st.set_page_config(
    page_title="🩺 Medical Chatbot - Gemini 1.5 Flash",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []
if "gemini_api_key" not in st.session_state:
    st.session_state.gemini_api_key = ""
if "model" not in st.session_state:
    st.session_state.model = None
if "uploaded_files_content" not in st.session_state:
    st.session_state.uploaded_files_content = ""
if "voice_recording" not in st.session_state:
    st.session_state.voice_recording = None

print("✅ Configuration and session state initialized!")

2025-09-12 04:05:50.604 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.605 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.606 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.607 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.608 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


✅ Configuration and session state initialized!


In [22]:
# Custom CSS for ChatGPT-like interface
st.markdown("""
<style>
    /* Hide Streamlit header and footer */
    .stApp > header {visibility: hidden;}
    #MainMenu {visibility: hidden;}
    footer {visibility: hidden;}

    /* Main container */
    .main-container {
        max-width: 1200px;
        margin: 0 auto;
        padding: 1rem;
    }

    /* Header styling */
    .chat-header {
        text-align: center;
        padding: 1rem 0;
        background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
        font-size: 2.5rem;
        font-weight: bold;
        margin-bottom: 2rem;
    }

    /* Chat input container - fixed at bottom */
    .chat-input-container {
        position: fixed;
        bottom: 0;
        left: 0;
        right: 0;
        background: white;
        padding: 1rem 2rem;
        border-top: 1px solid #e0e0e0;
        box-shadow: 0 -2px 10px rgba(0,0,0,0.1);
        z-index: 1000;
    }

    /* Input row layout */
    .input-row {
        display: flex;
        align-items: center;
        gap: 10px;
        max-width: 1000px;
        margin: 0 auto;
    }

    /* File upload button - left side */
    .file-upload-btn {
        background: #f0f2f6;
        border: 1px solid #d0d7de;
        border-radius: 50%;
        width: 40px;
        height: 40px;
        display: flex;
        align-items: center;
        justify-content: center;
        cursor: pointer;
        font-size: 18px;
        color: #666;
        transition: all 0.2s ease;
        user-select: none;
    }

    .file-upload-btn:hover {
        background: #e1e5e9;
        color: #333;
        transform: scale(1.05);
    }

    /* Voice button - right side */
    .voice-btn {
        background: #f0f2f6;
        border: 1px solid #d0d7de;
        border-radius: 50%;
        width: 40px;
        height: 40px;
        display: flex;
        align-items: center;
        justify-content: center;
        cursor: pointer;
        font-size: 16px;
        color: #666;
        transition: all 0.2s ease;
        user-select: none;
    }

    .voice-btn:hover {
        background: #e1e5e9;
        color: #333;
        transform: scale(1.05);
    }

    .voice-btn.recording {
        background: #ff4444 !important;
        color: white !important;
        animation: pulse 1s infinite;
    }

    @keyframes pulse {
        0% { transform: scale(1); }
        50% { transform: scale(1.1); }
        100% { transform: scale(1); }
    }

    /* Send button */
    .send-btn {
        background: #0066cc;
        color: white;
        border: none;
        border-radius: 50%;
        width: 40px;
        height: 40px;
        display: flex;
        align-items: center;
        justify-content: center;
        cursor: pointer;
        font-size: 18px;
        transition: all 0.2s ease;
        user-select: none;
    }

    .send-btn:hover {
        background: #0052a3;
        transform: scale(1.05);
    }

    /* Input field */
    .chat-input {
        flex: 1;
        padding: 12px 16px;
        border: 1px solid #d0d7de;
        border-radius: 25px;
        font-size: 16px;
        outline: none;
        width: 100%;
        background: white;
    }

    .chat-input:focus {
        border-color: #0066cc;
        box-shadow: 0 0 0 2px rgba(0, 102, 204, 0.2);
    }

    /* File indicator */
    .file-indicator {
        background: #e8f5e8;
        color: #2d5a2d;
        padding: 0.5rem 1rem;
        border-radius: 20px;
        font-size: 0.9rem;
        display: inline-block;
        margin: 0.5rem 0;
        border: 1px solid #c3e6c3;
    }

    /* Chat messages padding to avoid overlap with fixed input */
    .chat-messages {
        padding-bottom: 120px;
    }

    /* API key container */
    .api-key-container {
        background: #f8f9fa;
        padding: 1rem;
        border-radius: 10px;
        border: 1px solid #dee2e6;
        margin-bottom: 1rem;
    }

    /* Responsive design */
    @media (max-width: 768px) {
        .chat-input-container {
            padding: 0.75rem 1rem;
        }

        .input-row {
            gap: 8px;
        }

        .file-upload-btn, .voice-btn, .send-btn {
            width: 36px;
            height: 36px;
            font-size: 14px;
        }

        .chat-input {
            font-size: 14px;
            padding: 10px 14px;
        }
    }
</style>
""", unsafe_allow_html=True)

print("✅ CSS styling loaded!")

2025-09-12 04:05:50.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.626 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.627 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


✅ CSS styling loaded!


In [23]:
# JavaScript for Audio Recording and File Upload
AUDIO_RECORDER_JS = """
<script>
// Audio Recorder Class
class AudioRecorder {
    constructor() {
        this.mediaRecorder = null;
        this.audioChunks = [];
        this.isRecording = false;
    }

    async startRecording() {
        try {
            console.log('Starting audio recording...');
            const stream = await navigator.mediaDevices.getUserMedia({
                audio: {
                    sampleRate: 44100,
                    channelCount: 1,
                    volume: 1.0
                }
            });

            this.mediaRecorder = new MediaRecorder(stream, {
                mimeType: 'audio/webm'
            });
            this.audioChunks = [];

            this.mediaRecorder.ondataavailable = (event) => {
                if (event.data.size > 0) {
                    this.audioChunks.push(event.data);
                }
            };

            this.mediaRecorder.onstop = async () => {
                console.log('Recording stopped, processing audio...');
                const audioBlob = new Blob(this.audioChunks, { type: 'audio/webm' });
                const reader = new FileReader();
                reader.onloadend = () => {
                    const base64Audio = reader.result.split(',')[1];
                    console.log('Audio converted to base64, sending to Python...');
                    google.colab.kernel.invokeFunction('handle_audio_data', [base64Audio], {});
                };
                reader.readAsDataURL(audioBlob);
            };

            this.mediaRecorder.start();
            this.isRecording = true;

            // Update UI
            const voiceBtn = document.querySelector('.voice-btn');
            if (voiceBtn) {
                voiceBtn.classList.add('recording');
                voiceBtn.innerHTML = '⏹️';
            }

            console.log('Recording started successfully');

        } catch (error) {
            console.error('Error accessing microphone:', error);
            alert('Error accessing microphone: ' + error.message);
        }
    }

    stopRecording() {
        if (this.mediaRecorder && this.isRecording) {
            console.log('Stopping recording...');
            this.mediaRecorder.stop();

            // Stop all tracks
            if (this.mediaRecorder.stream) {
                this.mediaRecorder.stream.getTracks().forEach(track => track.stop());
            }

            this.isRecording = false;

            // Reset UI
            const voiceBtn = document.querySelector('.voice-btn');
            if (voiceBtn) {
                voiceBtn.classList.remove('recording');
                voiceBtn.innerHTML = '🎤';
            }
        }
    }

    toggleRecording() {
        if (this.isRecording) {
            this.stopRecording();
        } else {
            this.startRecording();
        }
    }
}

// File Upload Handler
function handleFileUpload() {
    console.log('File upload triggered');
    const input = document.createElement('input');
    input.type = 'file';
    input.multiple = true;
    input.accept = '.pdf,.txt,.docx,.png,.jpg,.jpeg,.doc';

    input.onchange = (event) => {
        const files = Array.from(event.target.files);
        console.log(`Selected ${files.length} files`);

        if (files.length === 0) return;

        const fileData = [];
        let processedFiles = 0;

        files.forEach((file, index) => {
            const reader = new FileReader();
            reader.onload = (e) => {
                fileData.push({
                    name: file.name,
                    type: file.type,
                    size: file.size,
                    content: e.target.result.split(',')[1]
                });

                processedFiles++;
                console.log(`Processed file ${processedFiles}/${files.length}: ${file.name}`);

                if (processedFiles === files.length) {
                    console.log('All files processed, sending to Python...');
                    google.colab.kernel.invokeFunction('handle_file_upload', [fileData], {});
                }
            };

            reader.onerror = (error) => {
                console.error(`Error reading file ${file.name}:`, error);
                processedFiles++;

                if (processedFiles === files.length) {
                    google.colab.kernel.invokeFunction('handle_file_upload', [fileData], {});
                }
            };

            reader.readAsDataURL(file);
        });
    };

    input.click();
}

// Message Submit Handler
function submitMessage() {
    const input = document.getElementById('chat-input');
    if (!input) {
        console.error('Chat input not found');
        return;
    }

    const message = input.value.trim();

    if (message) {
        console.log('Submitting message:', message);
        google.colab.kernel.invokeFunction('handle_chat_message', [message], {});
        input.value = '';
    }
}

// Initialize Audio Recorder
window.audioRecorder = new AudioRecorder();

// Global function for voice recording
function toggleRecording() {
    console.log('Toggle recording called');
    window.audioRecorder.toggleRecording();
}

// Event listeners
document.addEventListener('DOMContentLoaded', function() {
    // Handle Enter key on input
    const chatInput = document.getElementById('chat-input');
    if (chatInput) {
        chatInput.addEventListener('keypress', function(e) {
            if (e.key === 'Enter' && !e.shiftKey) {
                e.preventDefault();
                submitMessage();
            }
        });
    }
});

console.log('Audio recorder and file upload JavaScript loaded successfully');
</script>
"""

# Display the JavaScript
display(HTML(AUDIO_RECORDER_JS))
print("✅ Audio recording and file upload JavaScript loaded!")

✅ Audio recording and file upload JavaScript loaded!


In [24]:
# Helper Functions for Backend Processing

def configure_gemini(api_key: str):
    """Configure Gemini AI with API key"""
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-flash')
        return model
    except Exception as e:
        st.error(f"❌ Error configuring Gemini: {str(e)}")
        return None

def extract_text_from_pdf(file_content: bytes) -> str:
    """Extract text from PDF file"""
    try:
        pdf_file = io.BytesIO(file_content)
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

def extract_text_from_docx(file_content: bytes) -> str:
    """Extract text from DOCX file"""
    try:
        doc_file = io.BytesIO(file_content)
        doc = docx.Document(doc_file)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return text
    except Exception as e:
        return f"Error reading DOCX: {str(e)}"

def process_files(files_data: List[dict]) -> str:
    """Process uploaded files and extract text content"""
    combined_text = ""

    for file_data in files_data:
        try:
            file_name = file_data['name']
            file_type = file_data['type']
            file_content = base64.b64decode(file_data['content'])

            if file_type == "application/pdf":
                text = extract_text_from_pdf(file_content)
            elif file_type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
                text = extract_text_from_docx(file_content)
            elif file_type.startswith("text/"):
                text = file_content.decode('utf-8')
            elif file_type.startswith("image/"):
                text = f"[Image file: {file_name}]"
            else:
                text = f"[Unsupported file type: {file_name}]"

            combined_text += f"\n--- Content from {file_name} ---\n{text}\n"

        except Exception as e:
            combined_text += f"\n--- Error processing {file_name}: {str(e)} ---\n"

    return combined_text

def get_gemini_response(prompt: str, file_content: str = "") -> str:
    """Get response from Gemini with file context"""
    try:
        if not st.session_state.model:
            return "❌ Please configure your Gemini API key first."

        # Enhanced medical prompt
        enhanced_prompt = f'''You are a knowledgeable medical chatbot assistant. Provide accurate, helpful, and informative responses about medical topics.

**Important Guidelines:**
- Always remind users to consult healthcare professionals for serious medical concerns
- Provide evidence-based information when possible
- Be empathetic and supportive in your responses
- If you're uncertain about something, acknowledge it clearly
- Use simple, understandable language
- Include relevant medical terminology but explain complex terms

**User Query:** {prompt}

**Additional Context from uploaded files:**
{file_content if file_content else 'No additional files provided.'}

**Medical Assistant Response:**'''

        response = st.session_state.model.generate_content(enhanced_prompt)
        return response.text

    except Exception as e:
        return f"❌ Error generating response: {str(e)}"

def transcribe_audio(audio_base64: str) -> str:
    """Transcribe audio using speech recognition"""
    try:
        # Decode base64 audio
        audio_data = base64.b64decode(audio_base64)

        # Save as temporary file
        with tempfile.NamedTemporaryFile(suffix='.webm', delete=False) as tmp_file:
            tmp_file.write(audio_data)
            tmp_file_path = tmp_file.name

        # Convert to WAV format for speech recognition
        # Note: This is a simplified approach - in production you might need ffmpeg
        try:
            # Use speech recognition directly with the audio file
            r = sr.Recognizer()
            with sr.AudioFile(tmp_file_path) as source:
                audio = r.record(source)
                text = r.recognize_google(audio)
        except:
            # Fallback: treat as raw audio data
            audio_io = io.BytesIO(audio_data)
            r = sr.Recognizer()
            with sr.AudioFile(audio_io) as source:
                audio = r.record(source)
                text = r.recognize_google(audio)

        # Clean up
        os.unlink(tmp_file_path)

        return text

    except sr.UnknownValueError:
        return "Could not understand the audio"
    except sr.RequestError as e:
        return f"Speech recognition error: {e}"
    except Exception as e:
        return f"Error transcribing audio: {e}"

print("✅ Helper functions defined!")

✅ Helper functions defined!


In [25]:
# Colab Callback Functions for JavaScript Interaction

def handle_file_upload(files_data):
    """Handle file upload from JavaScript"""
    try:
        if files_data and len(files_data) > 0:
            # Process files and extract content
            file_content = process_files(files_data)
            st.session_state.uploaded_files_content = file_content

            # Show success message with file names
            file_names = [f['name'] for f in files_data]
            st.success(f"✅ Uploaded {len(files_data)} file(s): {', '.join(file_names)}")

            # Rerun to update UI
            st.rerun()

    except Exception as e:
        st.error(f"❌ Error handling file upload: {e}")

def handle_audio_data(audio_base64):
    """Handle audio recording from JavaScript"""
    try:
        if audio_base64:
            # Transcribe the audio
            transcribed_text = transcribe_audio(audio_base64)

            if transcribed_text and not transcribed_text.startswith("Error") and not transcribed_text.startswith("Could not"):
                # Store the transcribed text
                st.session_state.voice_recording = transcribed_text
                st.success(f"🎤 Voice input received: '{transcribed_text}'")

                # Auto-submit the voice message
                handle_chat_message(transcribed_text)
            else:
                st.warning(f"⚠️ {transcribed_text}")

    except Exception as e:
        st.error(f"❌ Error processing audio: {e}")

def handle_chat_message(message):
    """Handle chat message submission"""
    try:
        if not message.strip():
            return

        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": message})

        # Generate AI response
        with st.spinner("🤔 Thinking..."):
            response = get_gemini_response(
                message,
                st.session_state.uploaded_files_content
            )

            # Add assistant response to chat history
            st.session_state.messages.append({"role": "assistant", "content": response})

        # Clear voice recording state
        st.session_state.voice_recording = None

        # Rerun to update chat display
        st.rerun()

    except Exception as e:
        st.error(f"❌ Error processing message: {e}")

# Register Colab callback functions
output.register_callback('handle_file_upload', handle_file_upload)
output.register_callback('handle_audio_data', handle_audio_data)
output.register_callback('handle_chat_message', handle_chat_message)

print("✅ Colab callback functions registered!")

✅ Colab callback functions registered!


In [26]:
# Main Application UI

# Header
st.markdown('<div class="chat-header">🩺 Medical Chatbot - Gemini 1.5 Flash</div>', unsafe_allow_html=True)

# API Key Configuration
with st.expander("🔑 Gemini API Configuration", expanded=not st.session_state.model):
    st.markdown("Get your API key from: [Google AI Studio](https://makersuite.google.com/app/apikey)")

    col1, col2 = st.columns([3, 1])

    with col1:
        api_key = st.text_input(
            "Enter your Gemini API Key:",
            type="password",
            value=st.session_state.gemini_api_key,
            placeholder="AIzaSy..."
        )

    with col2:
        st.write("")  # Spacer
        if st.button("🔗 Connect", use_container_width=True):
            if api_key:
                model = configure_gemini(api_key)
                if model:
                    st.session_state.gemini_api_key = api_key
                    st.session_state.model = model
                    st.success("✅ Connected to Gemini!")
                    st.rerun()
            else:
                st.error("❌ Please enter a valid API key")

# System Status
col1, col2, col3 = st.columns(3)

with col1:
    if st.session_state.model:
        st.success("🟢 Gemini AI: Connected")
    else:
        st.warning("🟡 Gemini AI: Disconnected")

with col2:
    if st.session_state.uploaded_files_content:
        st.info("📎 Files: Attached")
    else:
        st.info("📎 Files: None")

with col3:
    st.info(f"💬 Messages: {len(st.session_state.messages)}")

print("✅ Main UI components loaded!")

2025-09-12 04:05:50.698 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.700 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.704 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.705 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.707 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.708 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

✅ Main UI components loaded!


In [27]:
# Chat Interface with Messages

# Chat messages container with proper spacing
st.markdown('<div class="chat-messages">', unsafe_allow_html=True)

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

st.markdown('</div>', unsafe_allow_html=True)

# File status indicator
if st.session_state.uploaded_files_content:
    st.markdown(
        '<div class="file-indicator">📎 Files attached and ready for medical analysis</div>',
        unsafe_allow_html=True
    )

# Voice recording status
if st.session_state.voice_recording:
    st.markdown(
        f'<div class="file-indicator">🎤 Voice input: "{st.session_state.voice_recording}"</div>',
        unsafe_allow_html=True
    )

print("✅ Chat interface ready!")

2025-09-12 04:05:50.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.765 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.767 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.768 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.768 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.769 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.772 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

✅ Chat interface ready!


In [28]:
# ChatGPT-like Input Interface with Voice and File Upload

# Custom input interface HTML - positioned at bottom like ChatGPT
input_interface_html = '''
<div class="chat-input-container">
    <div class="input-row">
        <!-- File Upload Button - Left side -->
        <div class="file-upload-btn" onclick="handleFileUpload()" title="Upload medical documents, images, or text files">
            📎
        </div>

        <!-- Main Input Field -->
        <div style="flex: 1; position: relative;">
            <input
                type="text"
                id="chat-input"
                class="chat-input"
                placeholder="Ask me anything about medical topics..."
                autocomplete="off"
            />
        </div>

        <!-- Voice Button - Right side -->
        <div class="voice-btn" onclick="toggleRecording()" title="Record voice message">
            🎤
        </div>

        <!-- Send Button -->
        <div class="send-btn" onclick="submitMessage()" title="Send message">
            ➤
        </div>
    </div>
</div>

<script>
// Ensure functions are available when DOM is ready
document.addEventListener('DOMContentLoaded', function() {
    console.log('Input interface loaded');

    // Focus on input when page loads
    const chatInput = document.getElementById('chat-input');
    if (chatInput) {
        chatInput.focus();

        // Handle Enter key
        chatInput.addEventListener('keypress', function(e) {
            if (e.key === 'Enter' && !e.shiftKey) {
                e.preventDefault();
                submitMessage();
            }
        });

        // Handle Escape key to clear input
        chatInput.addEventListener('keydown', function(e) {
            if (e.key === 'Escape') {
                chatInput.value = '';
            }
        });
    }
});

// Auto-focus input after each interaction
function focusInput() {
    setTimeout(() => {
        const input = document.getElementById('chat-input');
        if (input) {
            input.focus();
        }
    }, 100);
}

// Enhanced submit function with better UX
function submitMessage() {
    const input = document.getElementById('chat-input');
    if (!input) return;

    const message = input.value.trim();

    if (message) {
        // Disable input during processing
        input.disabled = true;
        input.placeholder = "Processing...";

        // Send message
        google.colab.kernel.invokeFunction('handle_chat_message', [message], {});

        // Clear and re-enable input
        input.value = '';
        setTimeout(() => {
            input.disabled = false;
            input.placeholder = "Ask me anything about medical topics...";
            focusInput();
        }, 1000);
    }
}

// Enhanced file upload with better feedback
function handleFileUpload() {
    console.log('Opening file dialog...');
    const input = document.createElement('input');
    input.type = 'file';
    input.multiple = true;
    input.accept = '.pdf,.txt,.docx,.doc,.png,.jpg,.jpeg,.csv';

    input.onchange = (event) => {
        const files = Array.from(event.target.files);

        if (files.length === 0) return;

        // Show upload feedback
        const fileBtn = document.querySelector('.file-upload-btn');
        if (fileBtn) {
            fileBtn.innerHTML = '⏳';
            fileBtn.style.backgroundColor = '#ffd700';
        }

        console.log(`Processing ${files.length} files...`);

        const fileData = [];
        let processedFiles = 0;

        files.forEach((file) => {
            // Check file size (max 10MB)
            if (file.size > 10 * 1024 * 1024) {
                console.warn(`File ${file.name} is too large (${file.size} bytes)`);
                processedFiles++;
                return;
            }

            const reader = new FileReader();
            reader.onload = (e) => {
                fileData.push({
                    name: file.name,
                    type: file.type,
                    size: file.size,
                    content: e.target.result.split(',')[1]
                });

                processedFiles++;

                if (processedFiles === files.length) {
                    console.log('All files processed, sending to backend...');
                    google.colab.kernel.invokeFunction('handle_file_upload', [fileData], {});

                    // Reset button
                    setTimeout(() => {
                        if (fileBtn) {
                            fileBtn.innerHTML = '📎';
                            fileBtn.style.backgroundColor = '#f0f2f6';
                        }
                    }, 2000);
                }
            };

            reader.onerror = () => {
                console.error(`Error reading file: ${file.name}`);
                processedFiles++;

                if (processedFiles === files.length && fileData.length > 0) {
                    google.colab.kernel.invokeFunction('handle_file_upload', [fileData], {});
                }
            };

            reader.readAsDataURL(file);
        });
    };

    input.click();
}

// Enhanced voice recording with better UI feedback
function toggleRecording() {
    if (window.audioRecorder) {
        const voiceBtn = document.querySelector('.voice-btn');

        if (window.audioRecorder.isRecording) {
            // Stop recording
            window.audioRecorder.stopRecording();
            if (voiceBtn) {
                voiceBtn.innerHTML = '🎤';
                voiceBtn.classList.remove('recording');
            }
        } else {
            // Start recording
            window.audioRecorder.startRecording();
            if (voiceBtn) {
                voiceBtn.innerHTML = '⏹️';
                voiceBtn.classList.add('recording');
            }
        }
    } else {
        console.error('Audio recorder not initialized');
        alert('Audio recorder not available. Please reload the page.');
    }
}

console.log('Enhanced input interface JavaScript loaded');
</script>
'''

# Display the input interface
st.components.v1.html(input_interface_html, height=120)

print("✅ ChatGPT-like input interface loaded!")

2025-09-12 04:05:50.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.795 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


✅ ChatGPT-like input interface loaded!


In [29]:
# Sidebar with Features and Settings

with st.sidebar:
    st.header("🛠️ Medical Chatbot Controls")

    # Quick Actions
    st.subheader("⚡ Quick Actions")

    col1, col2 = st.columns(2)

    with col1:
        if st.button("🗑️ Clear Chat", use_container_width=True):
            st.session_state.messages = []
            st.success("Chat cleared!")
            st.rerun()

    with col2:
        if st.button("📎 Remove Files", use_container_width=True):
            st.session_state.uploaded_files_content = ""
            st.success("Files removed!")
            st.rerun()

    # Statistics and Status
    st.subheader("📊 Session Statistics")

    # Create metrics
    messages_count = len(st.session_state.messages)
    user_messages = len([m for m in st.session_state.messages if m["role"] == "user"])
    ai_messages = len([m for m in st.session_state.messages if m["role"] == "assistant"])

    col1, col2 = st.columns(2)
    with col1:
        st.metric("Total Messages", messages_count)
        st.metric("User Messages", user_messages)

    with col2:
        st.metric("AI Responses", ai_messages)
        st.metric("Files Attached", "Yes" if st.session_state.uploaded_files_content else "No")

    # Features Status
    st.subheader("🎛️ Features Status")

    # API Connection Status
    if st.session_state.model:
        st.success("✅ Gemini AI: Connected")
    else:
        st.error("❌ Gemini AI: Not Connected")

    # File Upload Status
    if st.session_state.uploaded_files_content:
        st.success("✅ File Upload: Active")
        # Show file preview
        with st.expander("📄 View File Content"):
            st.text_area("File Content", st.session_state.uploaded_files_content, height=150)
    else:
        st.info("ℹ️ File Upload: No files attached")

    # Voice Recording Status
    st.success("✅ Voice Recording: Available")

    # Instructions
    st.subheader("💡 How to Use")

    st.markdown("""
    **1. Setup:**
    - Enter your Gemini API key above
    - Click "Connect" to activate AI

    **2. Chat:**
    - Type medical questions in the input box
    - Press Enter or click ➤ to send

    **3. Upload Files:**
    - Click 📎 to upload medical documents
    - Supports: PDF, DOCX, TXT, images

    **4. Voice Input:**
    - Click 🎤 to record voice messages
    - Speak clearly and click ⏹️ to stop

    **5. Medical Topics:**
    - Symptoms and conditions
    - Medication information
    - Health advice and tips
    - Medical document analysis
    """)

    # Safety Notice
    st.subheader("⚠️ Important Disclaimer")
    st.warning("""
    **Medical Disclaimer:**
    This chatbot is for educational and informational purposes only.

    **Always consult qualified healthcare professionals for:**
    - Medical diagnosis
    - Treatment decisions
    - Medication advice
    - Emergency situations

    **Do not use for life-threatening emergencies!**
    """)

    # Sample Questions
    st.subheader("💭 Sample Questions")

    sample_questions = [
        "What are the symptoms of diabetes?",
        "How does blood pressure medication work?",
        "What should I know about heart health?",
        "Explain the side effects of antibiotics",
        "How to maintain a healthy diet?"
    ]

    for i, question in enumerate(sample_questions):
        if st.button(f"💬 {question}", key=f"sample_{i}", use_container_width=True):
            # Add the sample question to chat
            st.session_state.messages.append({"role": "user", "content": question})

            # Generate AI response
            with st.spinner("🤔 Thinking..."):
                response = get_gemini_response(question, st.session_state.uploaded_files_content)
                st.session_state.messages.append({"role": "assistant", "content": response})

            st.rerun()

    # Footer
    st.markdown("---")
    st.markdown("""
    <div style="text-align: center; color: #666; font-size: 0.8em;">
        🩺 Medical Chatbot v1.0<br>
        Powered by Google Gemini AI<br>
        Built for Google Colab
    </div>
    """, unsafe_allow_html=True)

print("✅ Sidebar with features loaded!")

2025-09-12 04:05:50.853 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.864 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.866 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:50.869 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

✅ Sidebar with features loaded!


In [30]:
# Final Setup and Run Instructions

# Footer with information
st.markdown("---")

# Three column footer
col1, col2, col3 = st.columns(3)

with col1:
    st.markdown("**🤖 AI Model:** Google Gemini 1.5 Flash")

with col2:
    st.markdown("**💻 Platform:** Google Colab")

with col3:
    st.markdown("**✨ Features:** Voice + Files + Chat")

# Main footer
st.markdown("""
<div style="text-align: center; padding: 2rem; color: #666; background: #f8f9fa; border-radius: 10px; margin: 2rem 0;">
    <h3 style="color: #333; margin-bottom: 1rem;">🎉 Medical Chatbot Ready!</h3>

    <div style="display: flex; justify-content: center; gap: 2rem; flex-wrap: wrap; margin-bottom: 1rem;">
        <div style="text-align: center;">
            <div style="font-size: 2rem;">📎</div>
            <div><strong>File Upload</strong></div>
            <div style="font-size: 0.9em;">PDF, DOCX, Images</div>
        </div>

        <div style="text-align: center;">
            <div style="font-size: 2rem;">🎤</div>
            <div><strong>Voice Input</strong></div>
            <div style="font-size: 0.9em;">Speech-to-Text</div>
        </div>

        <div style="text-align: center;">
            <div style="font-size: 2rem;">🩺</div>
            <div><strong>Medical AI</strong></div>
            <div style="font-size: 0.9em;">Expert Responses</div>
        </div>

        <div style="text-align: center;">
            <div style="font-size: 2rem;">💬</div>
            <div><strong>Chat Interface</strong></div>
            <div style="font-size: 0.9em;">Like ChatGPT</div>
        </div>
    </div>

    <div style="font-size: 0.9em; line-height: 1.6;">
        Built with ❤️ using <strong>Streamlit</strong> & <strong>Google Gemini AI</strong><br>
        🩺 <em>For educational purposes - Always consult healthcare professionals</em>
    </div>
</div>
""", unsafe_allow_html=True)

# Success message
if st.session_state.model:
    st.balloons()
    st.success("🎉 Medical Chatbot is fully operational! Start chatting below.")
else:
    st.info("👆 Please configure your Gemini API key above to start using the chatbot")

# Auto-scroll to bottom (for better UX)
st.markdown("""
<script>
// Auto-scroll to bottom when new messages arrive
function scrollToBottom() {
    window.scrollTo(0, document.body.scrollHeight);
}

// Scroll after a short delay to ensure content is rendered
setTimeout(scrollToBottom, 500);
</script>
""", unsafe_allow_html=True)

print("✅ Medical Chatbot setup complete!")
print("🎉 All cells executed successfully!")

# Display final status
st.markdown("### 📋 Setup Status:")

setup_status = {
    "Dependencies": "✅ Installed",
    "Modules": "✅ Imported",
    "Configuration": "✅ Ready",
    "CSS Styling": "✅ Loaded",
    "JavaScript": "✅ Active",
    "Helper Functions": "✅ Defined",
    "Callback Functions": "✅ Registered",
    "UI Components": "✅ Rendered",
    "Chat Interface": "✅ Active",
    "Input Interface": "✅ ChatGPT-like",
    "Sidebar": "✅ Functional",
    "API Integration": "✅ Ready" if st.session_state.model else "⏳ Needs API Key"
}

# Display status in columns
col1, col2 = st.columns(2)

for i, (component, status) in enumerate(setup_status.items()):
    if i % 2 == 0:
        col1.write(f"**{component}:** {status}")
    else:
        col2.write(f"**{component}:** {status}")

2025-09-12 04:05:51.031 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:51.033 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:51.034 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:51.035 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:51.035 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:51.036 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:51.037 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 04:05:51.038 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

✅ Medical Chatbot setup complete!
🎉 All cells executed successfully!


In [31]:
# Get ngrok authentication token first
print("📝 Step 1: Get your ngrok authentication token")
print("🌐 Go to: https://dashboard.ngrok.com/get-started/your-authtoken")
print("📋 Copy your authtoken and paste it below")

# Install pyngrok
!pip install pyngrok -q

# Set your ngrok authtoken (replace with your actual token)
from pyngrok import ngrok

# ⚠️ REPLACE WITH YOUR ACTUAL TOKEN
NGROK_AUTHTOKEN = "32aD13X2Ko8T7rsmHcPNISs7MJg_2TCxTc9NztLE9Hga4gF1S"
ngrok.set_auth_token(NGROK_AUTHTOKEN)

print("✅ Ngrok authentication configured!")


📝 Step 1: Get your ngrok authentication token
🌐 Go to: https://dashboard.ngrok.com/get-started/your-authtoken
📋 Copy your authtoken and paste it below
✅ Ngrok authentication configured!


In [ ]:
# 🔧 ERR_NGROK_8012 COMPLETE FIX
import subprocess
import time
import os
from pyngrok import ngrok
from IPython.display import HTML, display

print("🔧 Fixing ERR_NGROK_8012 - Connection Refused Error")
print("=" * 60)

# Step 1: Kill ALL existing processes completely
print("🧹 Cleaning up ALL processes...")

!pkill -9 -f streamlit
!pkill -9 -f "python.*app"
!pkill -9 -f ngrok
ngrok.kill()

time.sleep(5)
print("✅ All processes terminated")

# Step 2: Verify port 8501 is available
print("🔍 Checking port 8501...")
port_check = !netstat -tuln | grep :8501
if port_check:
    print("⚠️ Port 8501 still in use, forcing cleanup...")
    !fuser -k 8501/tcp 2>/dev/null || true
    time.sleep(3)

print("✅ Port 8501 is now available")

# Step 3: Create working Streamlit app
print("📝 Creating medical chatbot...")

%%writefile /content/medical_app.py
import streamlit as st
import google.generativeai as genai
import os
import time

# Page config
st.set_page_config(
    page_title="🩺 Medical Chatbot - Gemini 1.5 Flash",
    page_icon="🤖",
    layout="wide"
)

# Custom CSS
st.markdown("""
<style>
    .main-header {
        text-align: center;
        background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        font-size: 3rem;
        font-weight: bold;
        margin-bottom: 2rem;
    }
    .status-success {
        background: #d4edda;
        border: 1px solid #c3e6cb;
        color: #155724;
        padding: 15px;
        border-radius: 10px;
        margin: 10px 0;
    }
    .status-error {
        background: #f8d7da;
        border: 1px solid #f5c6cb;
        color: #721c24;
        padding: 15px;
        border-radius: 10px;
        margin: 10px 0;
    }
</style>
""", unsafe_allow_html=True)

# Session state
if "messages" not in st.session_state:
    st.session_state.messages = []
if "model" not in st.session_state:
    st.session_state.model = None

def configure_gemini(api_key):
    """Configure Gemini AI"""
    try:
        if not api_key or len(api_key) < 20:
            return None
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-flash')
        # Test connection
        test_response = model.generate_content("Hello")
        return model
    except Exception as e:
        st.error(f"❌ Gemini error: {str(e)}")
        return None

def get_medical_response(prompt):
    """Generate medical response"""
    if not st.session_state.model:
        return "❌ Please configure your Gemini API key first."

    try:
        medical_prompt = f'''You are a medical AI assistant. Provide helpful medical information.

**Important:** Always recommend consulting healthcare professionals for diagnosis and treatment.

**Question:** {prompt}

**Response:**'''

        response = st.session_state.model.generate_content(medical_prompt)
        return response.text
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Header
st.markdown('<h1 class="main-header">🩺 Medical Chatbot - Gemini 1.5 Flash</h1>', unsafe_allow_html=True)

# Status indicators
col1, col2, col3 = st.columns(3)

with col1:
    if st.session_state.model:
        st.markdown('<div class="status-success">🟢 <strong>Gemini AI:</strong> Connected</div>', unsafe_allow_html=True)
    else:
        st.markdown('<div class="status-error">🔴 <strong>Gemini AI:</strong> Not Connected</div>', unsafe_allow_html=True)

with col2:
    st.markdown('<div class="status-success">🌐 <strong>Ngrok:</strong> Running</div>', unsafe_allow_html=True)

with col3:
    st.markdown(f'<div class="status-success">💬 <strong>Messages:</strong> {len(st.session_state.messages)}</div>', unsafe_allow_html=True)

# API Configuration
if not st.session_state.model:
    st.subheader("🔑 Configure Gemini API")
    st.info("Get your API key from: https://makersuite.google.com/app/apikey")

    col1, col2 = st.columns([3, 1])
    with col1:
        api_key = st.text_input("Gemini API Key:", type="password", placeholder="AIzaSy...")
    with col2:
        st.write("")
        if st.button("🔗 Connect", use_container_width=True):
            if api_key:
                with st.spinner("Connecting..."):
                    model = configure_gemini(api_key)
                    if model:
                        st.session_state.model = model
                        st.success("✅ Connected to Gemini!")
                        st.rerun()
            else:
                st.error("❌ Please enter API key")

# Chat Interface
if st.session_state.model:
    st.subheader("💬 Medical Chat")

    # Display messages
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.write(message["content"])

    # Chat input
    if user_input := st.chat_input("Ask medical questions..."):
        # Add user message
        st.session_state.messages.append({"role": "user", "content": user_input})
        with st.chat_message("user"):
            st.write(user_input)

        # Generate response
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                response = get_medical_response(user_input)
                st.write(response)
                st.session_state.messages.append({"role": "assistant", "content": response})

# Sidebar
with st.sidebar:
    st.header("🛠️ Controls")

    if st.button("🗑️ Clear Chat"):
        st.session_state.messages = []
        st.rerun()

    st.subheader("📊 Stats")
    st.metric("Messages", len(st.session_state.messages))

    st.subheader("💡 Sample Questions")
    samples = [
        "What are diabetes symptoms?",
        "How to lower blood pressure?",
        "Common cold treatment?"
    ]

    for i, q in enumerate(samples):
        if st.button(q, key=f"sample_{i}"):
            if st.session_state.model:
                st.session_state.messages.append({"role": "user", "content": q})
                response = get_medical_response(q)
                st.session_state.messages.append({"role": "assistant", "content": response})
                st.rerun()

    st.warning("⚠️ Always consult healthcare professionals!")

# Footer
st.markdown("---")
st.markdown("""
<div style="text-align: center; color: #666; padding: 1rem;">
    🩺 Medical Chatbot powered by Google Gemini AI<br>
    🌐 Running via Ngrok on Google Colab<br>
    <em>For educational purposes only</em>
</div>
""", unsafe_allow_html=True)

# Step 4: Start Streamlit with proper configuration
print("🚀 Starting Streamlit server...")

def start_streamlit():
    """Start Streamlit with proper configuration"""
    try:
        os.chdir('/content')

        cmd = [
            "streamlit", "run", "/content/medical_app.py",
            "--server.port", "8501",
            "--server.address", "0.0.0.0",
            "--server.headless", "true",
            "--server.enableCORS", "false",
            "--server.enableXsrfProtection", "false",
            "--browser.gatherUsageStats", "false"
        ]

        print(f"🔄 Command: {' '.join(cmd)}")

        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        return process

    except Exception as e:
        print(f"❌ Streamlit start error: {e}")
        return None

# Start Streamlit
streamlit_process = start_streamlit()

if streamlit_process:
    print("✅ Streamlit process created")

    # Wait for Streamlit to start
    print("⏳ Waiting for Streamlit to initialize...")

    for i in range(20):  # Wait up to 20 seconds
        time.sleep(1)

        # Check if process died
        if streamlit_process.poll() is not None:
            stdout, stderr = streamlit_process.communicate()
            print(f"❌ Streamlit died:")
            print(f"STDOUT: {stdout}")
            print(f"STDERR: {stderr}")
            break

        # Check if port is listening
        port_check = !netstat -tuln | grep :8501
        if port_check:
            print(f"✅ Port 8501 is listening! (attempt {i+1})")
            break

        print(f"⏳ Waiting... ({i+1}/20)")

    # Additional wait
    time.sleep(5)

    # Final port check
    final_check = !netstat -tuln | grep :8501
    if final_check:
        print("✅ CONFIRMED: Streamlit is running on port 8501")

        # Step 5: Create ngrok tunnel
        print("🌐 Creating ngrok tunnel...")

        try:
            ngrok.kill()
            time.sleep(2)

            public_url = ngrok.connect(8501)

            print("\n" + "🎉" + "="*60 + "🎉")
            print("✅ ERR_NGROK_8012 FIXED!")
            print("🌍 MEDICAL CHATBOT IS NOW LIVE!")
            print("="*64)
            print(f"🔗 PUBLIC URL: {public_url}")
            print("="*64)

            # Test connection
            print("🔍 Testing connection...")
            import requests
            try:
                response = requests.get(f"{public_url}", timeout=15)
                print(f"✅ Connection test: Status {response.status_code}")
            except Exception as e:
                print(f"⚠️ Connection test: {e} (tunnel may still work)")

            # Display success
            display(HTML(f'''
            <div style="
                background: linear-gradient(135deg, #28a745 0%, #20c997 100%);
                padding: 40px;
                border-radius: 20px;
                text-align: center;
                color: white;
                font-family: Arial, sans-serif;
                margin: 30px 0;
                box-shadow: 0 10px 30px rgba(0,0,0,0.3);
            ">
                <h1 style="margin: 0 0 20px 0; font-size: 2.8em;">✅ ERR_NGROK_8012 FIXED!</h1>
                <p style="font-size: 1.3em; margin: 0 0 20px 0;">Your medical chatbot is now running successfully!</p>

                <div style="
                    background: rgba(255,255,255,0.1);
                    padding: 20px;
                    border-radius: 15px;
                    margin: 20px 0;
                ">
                    <p style="font-size: 1.2em; margin: 0;">🌐 Public URL:</p>
                    <p style="font-size: 1.6em; margin: 10px 0; font-family: monospace;">
                        {public_url}
                    </p>
                </div>

                <a href="{public_url}" target="_blank" style="
                    display: inline-block;
                    background: rgba(255,255,255,0.9);
                    color: #333;
                    padding: 20px 40px;
                    border-radius: 30px;
                    text-decoration: none;
                    font-weight: bold;
                    font-size: 1.5em;
                    margin: 20px;
                ">
                    🚀 OPEN MEDICAL CHATBOT
                </a>

                <div style="margin-top: 30px;">
                    <h3>🔧 Problem Resolved:</h3>
                    <p>✅ Streamlit now running on port 8501</p>
                    <p>✅ Ngrok tunnel connected successfully</p>
                    <p>✅ No more connection refused errors</p>
                </div>
            </div>
            '''))

            print("🎯 SUCCESS! Your medical chatbot is accessible worldwide!")

        except Exception as e:
            print(f"❌ Ngrok error: {e}")
    else:
        print("❌ Streamlit is still not listening on port 8501")
        print("🔧 Try restarting the Colab runtime")
else:
    print("❌ Failed to start Streamlit")

print("🔄 Monitoring active...")


🔧 Fixing ERR_NGROK_8012 - Connection Refused Error
🧹 Cleaning up ALL processes...


In [34]:
# Monitor tunnel status
from pyngrok import ngrok
import time

def check_status():
    tunnels = ngrok.get_tunnels()
    if tunnels:
        print("🟢 Active Tunnels:")
        for tunnel in tunnels:
            print(f"   🌐 {tunnel.public_url}")
        return True
    else:
        print("🔴 No active tunnels")
        return False

def restart_tunnel():
    print("🔄 Restarting...")
    ngrok.kill()
    time.sleep(2)
    public_url = ngrok.connect(8501)
    print(f"✅ New URL: {public_url}")
    return public_url

def stop_all():
    print("🛑 Stopping all...")
    !pkill -f streamlit
    ngrok.kill()
    print("✅ Stopped")

# Status check
print("📊 Current Status:")
check_status()

# Keep alive
print(f"⏰ Status: {time.strftime('%H:%M:%S')}")

📊 Current Status:
🟢 Active Tunnels:
   🌐 https://26030978d81b.ngrok-free.app
⏰ Status: 04:06:52
